# Disentangled Multimodal Representation Learning for Recommendation (DMRL)

Ideias:
- FEITO: Usar a coluna para no texto
- FEITO: Limpar os dados de conteudo
- FEITO (lucas): ordenar os itens novos que ficaram por ultimo por popularidade
- FEITO: Adicionar o nome da coluna, porque vai indicar o assunto, em conjunto do texto

- tunar os hiperparametros
- usa a imagem alem do texto
- usar algum algoritmo mais simples apenas para usuarios novos, knn para content based
- Entender o parametro exclude_unknowns=True do RatioSplit e se tem alguma forma do DMRL gerar previsoes para itens e usuarios novos 

- paralelizar a previsao
Perguntas:


## Imports

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import torch
import cornac
import numpy as np
import pandas as pd
from cornac.metrics import NDCG
from cornac.eval_methods import RatioSplit
from cornac.data import TextModality, ImageModality
from cornac.models.dmrl.recom_dmrl import DMRL 
from tqdm import tqdm
from utils import load_data, preprocessing_content_data
import requests
from PIL import Image
from io import BytesIO
from joblib import Parallel, delayed


# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
np.set_printoptions(threshold=np.inf)

/home/matheus/miniconda/envs/sistema_recomendacao_tp2_versao3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load and process data

### Load data

In [4]:
ratings, content, targets = load_data()

In [5]:
ratings["TimestampDate"] = ratings['Timestamp'].dt.date
ratings.loc[ratings.Rating == 0, "Rating"] = 0.01

### Limpar os dados de conteudo

- FEITO - Year: deletar o - no texto '2013–'
- FEITO - Rated: alterar as diversas formas de escrever NA para NA. Esse e um caso especial
- FEITO - alterar as diversas formas de escrever NA para None em todas as coluans
- FEITO - Language: tem varias linhas que possuem bizarices como  'None, English' e 'None, French' , 'English, None'...
- FEITO - Ratings: criar uma coluna para cada chave do dicionario, entender quais sao todas as chaves que existem

-----------------------------------------------
Variaveis que nao precisariam ser tratadas com um bert:
- Metascore
- imdbRating
- Type

In [6]:
content_auxiliar = content.drop(columns=["Poster", "Website", "Response", "Episode", "seriesID"]).copy()

In [7]:
content_auxiliar['Year'] = content_auxiliar['Year'].str.replace('–', '')

In [8]:
nan = content.totalSeasons.unique()[0]
dict_transform_to_na = {
    "Rated":['N/A', 'Not Rated', 'Unrated', 'UNRATED', 'NOT RATED'],
    "all": [nan, 'N/A', 'None', np.nan],
}

for na_value in dict_transform_to_na["all"]:
    content_auxiliar = content_auxiliar.replace(na_value, None)

for na_value in dict_transform_to_na["Rated"]:
    content_auxiliar['Rated'] = content_auxiliar['Rated'].replace(na_value, None)

In [9]:
content_auxiliar['Language'] = content_auxiliar['Language'].str.replace('None, ', '')
content_auxiliar['Language'] = content_auxiliar['Language'].str.replace(', None', '')

In [10]:
# Entendendo os valores possiveis para a coluna Ratings
# A coluna content_auxiliar.Ratings quarda uma lista que posde ter entre 0 e 3 dicionarios. Cada dicionario possui a chave 'Source', 'Value'.
num_ratings_per_item = []
unique_keys = []
rating_sources = []
rating_values = []

for rating_list in content_auxiliar.Ratings:
    num_ratings_per_item.append(len(rating_list))
    for rating_dict in rating_list:
        for key in rating_dict:
            unique_keys.append(key)
        rating_sources.append(rating_dict['Source'])
        rating_values.append(rating_dict['Value'])

set(rating_sources)

{'Internet Movie Database', 'Metacritic', 'Rotten Tomatoes'}

In [11]:
InternetMovieDatabase_list = []
Metacritic_list = []
RottenTomatoes_list = []
for rating_list in content_auxiliar.Ratings:
    InternetMovieDatabase_list.append(None)
    Metacritic_list.append(None)
    RottenTomatoes_list.append(None)
    for rating_dict in rating_list:
        if rating_dict['Source'] == 'Internet Movie Database':
            InternetMovieDatabase_list[-1] = rating_dict['Value']
        elif rating_dict['Source'] == 'Metacritic':
            Metacritic_list[-1] = rating_dict['Value']
        elif rating_dict['Source'] == 'Rotten Tomatoes':
            RottenTomatoes_list[-1] = rating_dict['Value']

In [12]:
content_auxiliar['Internet Movie Database'] = InternetMovieDatabase_list
content_auxiliar['Metacritic'] = Metacritic_list
content_auxiliar['Rotten Tomatoes'] = RottenTomatoes_list


In [13]:
content_auxiliar.drop(columns=['Ratings'], inplace=True)

### Apendar a coluna no valor do dataframe

In [14]:
content_columns = content_auxiliar.columns.to_list()
content_columns.pop(0)

'ItemId'

In [15]:
for column in content_columns:
    content_auxiliar[column] = content_auxiliar[column].apply(lambda x: f"{column}: {x}; " if x is not None else f"{column}: unknown value; ")

In [16]:
content_processed = content_auxiliar[['ItemId']].copy()
content_processed["text"] = content_auxiliar[content_columns].astype(str).fillna('').agg(' '.join, axis=1)

### Pegando os dados de imagem

In [17]:
# folder_path = 'poster_images'

# # Lista todos os arquivos na pasta
# file_list = os.listdir(folder_path)
# item_id_saved = [file.split('=')[1].split('.')[0] for file in file_list]

In [18]:
# content_to_get_image = content.loc[content.Poster != 'N/A', ['ItemId', 'Poster']].copy()
# content_to_get_image = content_to_get_image.loc[~content_to_get_image.ItemId.isin(item_id_saved), :]
# content_to_get_image = content_to_get_image.reset_index(drop=True)

In [19]:
# def download_image(index):
#     image_url = content_to_get_image.loc[index, "Poster"]
#     item_id = content_to_get_image.loc[index, "ItemId"]

#     # Fazer a requisição HTTP para baixar a imagem
#     # try:
#     response = requests.get(image_url)
#     # except:
#         # return None

#     # Verificar se a requisição foi bem-sucedida
#     if response.status_code == 200:
#         # Abrir a imagem
#         img = Image.open(BytesIO(response.content))
#         # Salvar a imagem
#         img.save(f"{folder_path}/item_id={item_id}.jpg")
#         return None
#     else:
#         return (item_id, image_url)

# # Lista de índices para processar
# indices = content_to_get_image.index.tolist()

# # Usar todos os CPUs disponíveis
# result_list = Parallel(n_jobs=-1, verbose=100)(delayed(download_image)(index) for index in indices)

# # Filtrar os resultados que não conseguiram baixar
# nao_conseguiu_baixar = [result for result in result_list if result is None]

## Train model

### Processamento de imagem

In [20]:
# from PIL import Image
# import numpy as np
# import os

# folder_path = 'poster_images'
# image_files = os.listdir(folder_path)
# images_item_ids = [file.split('=')[1].split('.')[0] for file in image_files]

# # Initialize a list to store the image matrices
# def process_image(image_file):
#     image_path = os.path.join(folder_path, image_file)
#     image = Image.open(image_path)
#     image_matrix = np.array(image)
#     return image_matrix.reshape(-1)

# # Use Parallel to process images in parallel
# image_matrices = Parallel(n_jobs=-1, verbose=100)(delayed(process_image)(image_file) for image_file in image_files)

In [21]:
#  E preciso que todas as imagens tenham o mesmo tamanho, para isso vamos adicionar zeros no final das imagens menores

# maior_imagem = -1
# for image in image_matrices:
#     if image.shape[0] > maior_imagem:
#         maior_imagem = image.shape[0]

# for index, image in enumerate(image_matrices):
#     new_image = np.insert(image, len(image), np.zeros(maior_imagem - len(image)))
#     del image
#     image_matrices[index] = new_image

# def add_zeros_to_image(image):
#     new_image = np.insert(image, len(image), np.zeros(maior_imagem - len(image)))
#     del image
#     return new_image

# image_matrices = Parallel(n_jobs=-1, verbose=100)(delayed(add_zeros_to_image)(image) for image in image_matrices)

In [22]:
#  E preciso que todas as imagens tenham o mesmo tamanho, para isso vamos deletar as maoires imagens
# menor_imagem = np.Inf
# for image in image_matrices:
#     if image.shape[0] < menor_imagem:
#         menor_imagem = image.shape[0]

# def diminui_a_imagem(image):
#     return image[:menor_imagem]

# image_matrices2 = Parallel(n_jobs=-1, verbose=100)(delayed(diminui_a_imagem)(image) for image in image_matrices)

# Convert the list of image matrices to a numpy array
# image_matrices2 = np.array(image_matrices2)
# image_modality = ImageModality(features=image_matrices2, ids=images_item_ids)

### Formato do dado de texto

In [23]:
item_text_modality = TextModality(
    corpus=content_processed.text.to_list(),
    ids=content_processed.ItemId.to_list(),
)

### Otimizacao de hiperparametros

In [24]:
from skopt import dummy_minimize

In [39]:
import dill

def save_object(obj, filename):
    with open(filename, 'wb') as file:
        dill.dump(obj, file)

def load_object(filename):
    with open(filename, 'rb') as file:
        return dill.load(file)

In [44]:
ratio_split = RatioSplit(
    data=ratings[['UserId', 'ItemId', 'Rating']].values.tolist(),
    test_size=0.2,
    exclude_unknowns=True,
    verbose=True,
    seed=12012001,
    rating_threshold=0.5,
    item_text=item_text_modality
)

rating_threshold = 0.5
exclude_unknowns = True
---
Training data:
Number of users = 46750
Number of items = 27045
Number of ratings = 527776
Max rating = 10.0
Min rating = 0.0
Global mean = 7.3
---
Test data:
Number of users = 46750
Number of items = 27045
Number of ratings = 124031
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 46750
Total items = 27045


In [25]:
def DMR_pipeline(parameters):
    learning_rate, decay_c, decay_r, num_factors = parameters
    # learning_rate, decay_c, decay_r, epochs, embedding_dim, bert_text_dim, num_factors = parameters
    print("Parametros da minimizacao" + "="*150)
    model_parameters = (
        # f"learning_rate={learning_rate},decay_c={decay_c},decay_r={decay_r},embedding_dim={embedding_dim},bert_text_dim={bert_text_dim},num_factor={num_factors}"
        f"learning_rate={learning_rate},decay_c={decay_c},decay_r={decay_r},num_factor={num_factors}"

        )
    print(model_parameters)

    dmrl_recommender = DMRL(
        batch_size=1024,
        epochs=20,
        learning_rate=learning_rate,
        decay_c=decay_c,
        decay_r=decay_r,
        # epochs=epochs,
        # embedding_dim=embedding_dim,
        # bert_text_dim=bert_text_dim,
        num_factors=num_factors,
        verbose=True,
    )
    experiment = cornac.Experiment(
        eval_method=ratio_split,
        models=[dmrl_recommender], metrics=[NDCG(20)]
    )
    
    try:
        print("Iniciando o experimento...")
        experiment_returned = experiment.run()
        metric = experiment.result[0].metric_avg_results['NDCG@20']
        print("Metrica NDCG@20: ", metric)
        model_path = "dmrl_models/" + f"NDCG@20={metric}," + model_parameters + ".model"
        save_object(dmrl_recommender, model_path)
        # dmrl_recommender.save("dmrl_models/" + f"NDCG@20={metric}," + model_parameters + ".model")
        print("Fim do experimento", "="*50)
        return -1 * metric
    except RuntimeError as e:
        print("Erro na execução do experimento, retornando Infinito")
        print(e)
        print("Fim do experimento", "="*100)
        return np.Inf

In [26]:
space = [
    (0.00001, 0.1),  # learning_rate
    (0.001, 1),  # decay_c # comentar isso nao resolveu o problema do cuda
    (0.0001, 0.1),  # decay_r # comentar isso nao resolveu o problema do cuda
    # [20, 30, 40, 50], # epochs # comentar isso nao resolveu o problema do cuda
    # [40, 128, 200, 300, 500],  # embedding_dim # tem relacao com o erro de dimensao de multiplicacao de matriz
    # [100, 383, 600, 800, 1000],  # bert_text_dim # tem relacao com o erro de dimensao de multiplicacao de matriz
    [2, 4, 8],  # num_factors # comentar isso nao resolveu o problema do cuda
]

In [27]:
# result = dummy_minimize(DMR_pipeline, space, verbose=True, n_calls=30, random_state=12012001)

### Le o melhor modelo

In [43]:
model = load_object('dmrl_models/NDCG@20=0.08492874336076696,learning_rate=0.09938198721604344,decay_c=0.8755381083510204,decay_r=0.0004231976548177414,num_factor=2.model')

## Preve o modelo para os dados de treino

In [63]:
from tqdm import tqdm

In [ ]:
user_id_train = ratings.UserId.unique()

In [ ]:
ratings_prediction = ratings.copy()
ratings_prediction["PredictRating"] = -1

for user_id in tqdm(user_id_train):
    # Get the train dataframe index of the user to predict
    user_index = ratio_split.train_set.uid_map.get(user_id)

    if user_index is None:
        # print(f"User {user_id} is not in the train set")
        continue

    # Flter by items to predict 
    items_to_predict = ratings.loc[ratings.UserId == user_id, "ItemId"].to_list()

    # Get the train dataframe index of the items to predict
    items_to_predict_index = np.array([ratio_split.train_set.iid_map.get(item_id) for item_id in items_to_predict])

    items_to_predict_tensor = torch.tensor([idx for idx in items_to_predict_index if idx is not None])

    # Get the position of items that are not in the train set
    none_indices = [i for i, x in enumerate(items_to_predict_index) if x is None]

    # Get the prediction for the items
    line_rating = model.score(user_index=user_index, item_indices=items_to_predict_tensor)

    # Insert -1 in the position of items that are not in the train set
    for index_to_insert in none_indices:
        line_rating = np.insert(line_rating, index_to_insert, -1)

    # Insert the prediction in the ratings_prediction dataframe
    ratings_prediction.loc[ratings.UserId == user_id, "PredictRating"] = line_rating

  0%|          | 13/51671 [00:00<54:09, 15.90it/s] 

User 1f0e3dad99 is not in the train set


  0%|          | 26/51671 [00:01<46:31, 18.50it/s]

User a5bfc9e079 is not in the train set


  0%|          | 29/51671 [00:01<55:16, 15.57it/s]


ValueError: Must have equal len keys and value when setting with an iterable

## Preve para os dados de teste

In [45]:

target_prediction = targets.copy()
ratings_prediction["Rating"] = -1

user_id_list = targets.UserId.unique()
for user_id in user_id_list:
    # Get the train dataframe index of the user to predict
    user_index = ratio_split.train_set.uid_map.get(user_id)

    if user_index is None:
        print(f"User {user_id} is not in the train set")
        continue

    # Flter by items to predict 
    items_to_predict = targets.loc[targets.UserId == user_id, "ItemId"].to_list()

    # Get the train dataframe index of the items to predict
    items_to_predict_index = np.array([ratio_split.train_set.iid_map.get(item_id) for item_id in items_to_predict])

    items_to_predict_tensor = torch.tensor([idx for idx in items_to_predict_index if idx is not None])

    # Get the position of items that are not in the train set
    none_indices = [i for i, x in en
                    
                    umerate(items_to_predict_index) if x is None]

    # Get the prediction for the items
    line_rating = model.score(user_index=user_index, item_indices=items_to_predict_tensor)

    # Insert -1 in the position of items that are not in the train set
    for index_to_insert in none_indices:
        line_rating = np.insert(line_rating, index_to_insert, -1)

    # Insert the prediction in the ratings_prediction dataframe
    target_prediction.loc[targets.UserId == user_id, "Rating"] = line_rating

User 02b780c583 is not in the train set
User 03fbe3c1e8 is not in the train set
User 0503bf6097 is not in the train set
User 0958560bd4 is not in the train set
User 0b17c3df40 is not in the train set
User 10c3aef33f is not in the train set
User 119653373a is not in the train set
User 139ccdbf5a is not in the train set
User 1671fa4df3 is not in the train set
User 16757f4d90 is not in the train set
User 187454feb4 is not in the train set
User 1af811e7b9 is not in the train set
User 1bbf59e4a4 is not in the train set
User 1ef563bb8b is not in the train set
User 1f6a565547 is not in the train set
User 211a7a84d3 is not in the train set
User 243be2818a is not in the train set
User 24a13b72fc is not in the train set
User 27649dc46e is not in the train set
User 2a3500f75b is not in the train set
User 2ae6906d5d is not in the train set
User 2cfdb14e05 is not in the train set
User 2ed4d0c23c is not in the train set
User 2ef13886ae is not in the train set
User 302c6ebc7d is not in the train set


In [46]:
target_prediction = target_prediction.sort_values(["UserId", "Rating"], ascending=[True, False])

In [49]:
target_prediction.to_csv("submissoes/DMRL_versao_6.csv", index=False)

In [50]:
target_prediction = target_prediction.drop(columns="Rating")

In [51]:
target_prediction.to_csv("submissoes/DMRL_versao_6_sem_rating.csv", index=False)